In [33]:
# ruff: noqa: N802, N803, N806, N815, N816
import os
import pprint

import numpy as np
from scipy import signal

# Simple utilities for displaying generated code in the notebook
from utils import cleanup, display_text

import archimedes as arc
from archimedes import struct

THEME = os.environ.get("ARCHIMEDES_THEME", "dark")
arc.theme.set_theme(THEME)

# The Codegen API

In the previous part of this tutorial, we learned how to automatically translate Python code to highly efficient C implementations.
However, the C code generated by CasADi uses a generic interface that requires that all arrays be initialized externally.
If we want to call the generated function from our own C code or use it in an embedded setting, this creates overhead of having to manually determine and adjust array sizes, names, initializations, etc.

In embedded systems, memory management is critical. The "kernel" implementation of the algorithm is extremely efficient, but it expects:

1. Pre-allocated memory buffers for all inputs and outputs
2. Properly configured pointers to these buffers
3. Initialization of working memory needed by the algorithm

Manually handling these details would be tedious and error-prone - especially when algorithms change during development.

Archimedes handles this with an "interface layer", which creates a predictable and easy-to-use API for working with the low-level kernel code.  On this page we will go through some of the details of this interface layer.

First, let's recall the basic IIR filter implementation we were working with:

In [11]:
# Note: give descriptive names for return values (these don't need
# to match the variable names, but must be different from arg names)
@arc.compile(return_names=("u_hist", "y_hist"))
def iir_filter(u, b, a, u_prev, y_prev):
    # Update input history
    u_prev[1:] = u_prev[:-1]
    u_prev[0] = u

    # Compute output using the direct II transposed structure
    y = (np.dot(b, u_prev) - np.dot(a[1:], y_prev[: len(a) - 1])) / a[0]

    # Update output history
    y_prev[1:] = y_prev[:-1]
    y_prev[0] = y

    return u_prev, y_prev

We will generate the code just as before:

In [12]:
cleanup()  # Clean up any previous generated code

# Design a simple IIR filter with SciPy
dt = 0.01  # Sampling time [seconds]
Wn = 10  # Cutoff frequency [Hz]
order = 4
b, a = signal.butter(order, Wn, "low", analog=False, fs=1 / dt)

# Create "template" arguments for type inference
u = 1.0
u_prev = np.zeros(len(b))
y_prev = np.zeros(len(a) - 1)
args = (u, b, a, u_prev, y_prev)

arc.codegen(iir_filter, args)

## The interface code

Last time we looked at the kernel code, which used flattened arrays of pointers to access inputs, outputs, and working memory.
While highly efficient, this code on its own is difficult to work with directly, particularly for more complex functions with multiple arguments and return values.

Here's what the "interface" code looks like:

In [13]:
with open("iir_filter.h", "r") as f:
    c_code = f.read()

display_text(c_code)

```c

#ifndef IIR_FILTER_H
#define IIR_FILTER_H

#include "iir_filter_kernel.h"

#ifdef __cplusplus
extern "C" {
#endif

// Input arguments struct
typedef struct {
    float u;
    float b[5];
    float a[5];
    float u_prev[5];
    float y_prev[4];
} iir_filter_arg_t;

// Output results struct
typedef struct {
    float u_hist[5];
    float y_hist[4];
} iir_filter_res_t;

// Workspace struct
typedef struct {
    long int iw[iir_filter_SZ_IW];
    float w[iir_filter_SZ_W];
} iir_filter_work_t;

// Runtime API
int iir_filter_init(iir_filter_arg_t* arg, iir_filter_res_t* res, iir_filter_work_t* work);
int iir_filter_step(iir_filter_arg_t* arg, iir_filter_res_t* res, iir_filter_work_t* work);


#ifdef __cplusplus
}
#endif

#endif // IIR_FILTER_H
```

Now, instead of manually wrangling pointer arrays, we have function-specific input and output structs with meaningful names.

The header is enough for us to work with this in our own code, but for completeness here's the corresponding C code:

In [14]:
with open("iir_filter.c", "r") as f:
    c_code = f.read()

display_text(c_code)

```c

#include <string.h>
#include "iir_filter.h"

int iir_filter_init(iir_filter_arg_t* arg, iir_filter_res_t* res, iir_filter_work_t* work) {
    if (!arg || !res || !work) {
        return -1; // Invalid pointers
    }

    /* Initialize inputs */
    memset(arg, 0, sizeof(iir_filter_arg_t));

    /* Initialize outputs */
    memset(res, 0, sizeof(iir_filter_res_t));

    /* Nonzero assignments */
    arg->u = 1.000000f;
    arg->b[0] = 0.004824f;
    arg->b[1] = 0.019297f;
    arg->b[2] = 0.028946f;
    arg->b[3] = 0.019297f;
    arg->b[4] = 0.004824f;
    arg->a[0] = 1.000000f;
    arg->a[1] = -2.369513f;
    arg->a[2] = 2.313988f;
    arg->a[3] = -1.054665f;
    arg->a[4] = 0.187379f;

    return 0;
}

int iir_filter_step(iir_filter_arg_t* arg, iir_filter_res_t* res, iir_filter_work_t* work) {
    if (!arg || !res || !work) {
        return -1; // Invalid pointers
    }

    // Marshal inputs to CasADi format
    const float* kernel_arg[iir_filter_SZ_ARG];
    kernel_arg[0] = &arg->u;
    kernel_arg[1] = arg->b;
    kernel_arg[2] = arg->a;
    kernel_arg[3] = arg->u_prev;
    kernel_arg[4] = arg->y_prev;
    
    // Marshal outputs to CasADi format
    float* kernel_res[iir_filter_SZ_RES];
    kernel_res[0] = res->u_hist;
    kernel_res[1] = res->y_hist;
    
    // Call kernel function
    return iir_filter(kernel_arg, kernel_res, work->iw, work->w, 0);
}
```

If you are familiar with C, this code will be largely self-explanatory.
It takes care of all array and pointer initialization, specifically initializing all arrays to the values of the "template arguments" we passed to `codegen`.

One additional piece that's worth noting: the kernel code had some nice properties including deterministic and static memory allocation, and portable, self-contained code.  The interface layer preserves these features, so it is also suitable for either standalone C/C++ applications or deployment to embedded platforms.

## Working with the generated API

In order to use the generated code we have to do three things:

```c
#include "iir_filter.h"

// 1. Declare the input, output, and workspace structures
iir_filter_arg_t arg;
iir_filter_res_t res;
iir_filter_work_t work;

// 2. Initialize the structs
iir_filter_init(&arg, &res, &work);

// 3. Call the function
iir_filter_step(&arg, &res, &work);
```

Note that, as in the "quick start" example earlier in this series, for recursive filters like the IIR, the `iir_filter_step` function does not place the outputs back in the input arrays.
For example, a more useful iteration as part of a timed interrupt on an STM32 platform might look like the following:

```c
int n = sizeof(arg.b) / sizeof(arg.b[0]) - 1;  // Filter order
float y;  // Output, to be fed to actuator, control algorithm, etc.

void HAL_TIM_PeriodElapsedCallback(TIM_HandleTypeDef *htim) {
    if (htim->Instance == TIM3) {
        arg.u = dma_buffer[0];  // Read from sensor, other algorithm output, etc.
        iir_filter_step(&arg, &res, &work);
        y = res.y_hist[0];  // Output, to be fed to actuator, control algorithm, etc.

        // Copy output arrays back to inputs
        for (int j = 0; j < n; j++) {
            arg.u_prev[j] = res.u_hist[j];
            arg.y_prev[j] = res.y_hist[j];
        }
        arg.u_prev[n] = res.u_hist[n];
    }
}
```

As a side note, you may well notice here that we're using roughly twice as much memory as we strictly need, since in this case we know that the outputs will be fed back to the inputs.
This will generally be the case for recursive structures (IIR filters, sensor fusion, and similar algorithms) and highlights a limitation of the "functionally pure" assumption underlying code generation in Archimedes (and CasADi).
While the generated code is efficient, it is not necessarily optimal; if you are working with tightly resource-constrained platforms or otherwise need extreme performance, you may still need hand-coding or other specialized tools.

On the other hand, if you can afford to keep a few extra floats in memory and perform the associated copy operations, this workflow provides a clean way to abstract the algorithm implementation from the rest of the embedded application.

For example, if we want to choose a different order and cutoff filter, we can do so easily:

In [15]:
cleanup()  # Cleanup any previous runs

# Design a simple IIR filter with SciPy
dt = 0.01  # Sampling time [seconds]
Wn = 20  # Cutoff frequency [Hz]
order = 2
b, a = signal.butter(order, Wn, "low", analog=False, fs=1 / dt)

# Create "template" arguments for type inference
u = 1.0
u_prev = np.zeros(len(b))
y_prev = np.zeros(len(a) - 1)
args = (u, b, a, u_prev, y_prev)

arc.codegen(iir_filter, args)

with open("iir_filter.h", "r") as f:
    c_code = f.read()

display_text(c_code)

```c

#ifndef IIR_FILTER_H
#define IIR_FILTER_H

#include "iir_filter_kernel.h"

#ifdef __cplusplus
extern "C" {
#endif

// Input arguments struct
typedef struct {
    float u;
    float b[3];
    float a[3];
    float u_prev[3];
    float y_prev[2];
} iir_filter_arg_t;

// Output results struct
typedef struct {
    float u_hist[3];
    float y_hist[2];
} iir_filter_res_t;

// Workspace struct
typedef struct {
    long int iw[iir_filter_SZ_IW];
    float w[iir_filter_SZ_W];
} iir_filter_work_t;

// Runtime API
int iir_filter_init(iir_filter_arg_t* arg, iir_filter_res_t* res, iir_filter_work_t* work);
int iir_filter_step(iir_filter_arg_t* arg, iir_filter_res_t* res, iir_filter_work_t* work);


#ifdef __cplusplus
}
#endif

#endif // IIR_FILTER_H
```

This has an identical API to the first version and will automatically initialize the correct filter coefficients, meaning we can tune the filter and re-deploy without changing any of the application code!

### Platform Portability

Similar usage patterns can be applied in a range of contexts; since the generated code is self-contained it is highly portable (provided the compiler supports ANSI C).

For example, here's a "cookbook recipe" for running a generated function in a timed loop on an Arduino.

In [16]:
with open("iir.ino", "r") as f:
    c_code = f.read()

display_text(c_code)

```c
#include <Arduino.h>
#include <TimerOne.h>
#include "iir_filter.h"

// Sampling rate: 100 Hz
const unsigned long SAMPLE_RATE_US = 10000;

// Declare the input, output, and workspace structures
iir_filter_arg_t arg;
iir_filter_res_t res;
iir_filter_work_t work;

int n = sizeof(arg.b) / sizeof(arg.b[0]) - 1;  // Filter order
float y;  // Output, to be fed to actuator, control algorithm, etc.

// Timer interrupt handler
void timerInterrupt() {
    arg.u = 1.0;  // Or read from sensor, other algorithm output, etc.
    iir_filter_step(&arg, &res, &work);  // Call IIR filter
    y = res.y_hist[0];  // Output, to be fed to actuator, control algorithm, etc.

    // Copy output arrays back to inputs
    for (int j = 0; j < n; j++) {
        arg.u_prev[j] = res.u_hist[j];
        arg.y_prev[j] = res.y_hist[j];
    }
    arg.u_prev[n] = res.u_hist[n];
}

void setup(){
    // Initialize the structs
    iir_filter_init(&arg, &res, &work);

    Serial.begin(9600);

    // Initialize Timer1 for interrupts at 100 Hz
    Timer1.initialize(SAMPLE_RATE_US);
    Timer1.attachInterrupt(timerInterrupt);
}

void loop() {
    // ...non-time-critical tasks
    delay(10);
}
```

Note that the filtering code is identical to the STM32 snippet; the only differences here are in peripheral configuration and other platform-specific HAL code.
The same auto-generated code can target everything from rapid prototyping platforms to production-grade embedded controllers to Unix clusters.

## PyTree arguments and returns

We started for simplicity with a function that operates with regular arrays, but this doesn't scale well to more complex algorithms.
Even if we added a second filter, we might need as many as ten arguments to keep track of the filter histories, coefficients, etc.
This quickly becomes difficult to understand and maintain.

One solution is to use hierarchical data structures to organize the arguments and returns.
We call these "PyTrees", using terminology borrowed from [JAX](https://docs.jax.dev/en/latest/pytrees.html).
These let you organize state and parameters in dataclass-like containers which can be mapped directly to a C `struct`.
They can even be nested inside of one another to arbitrary depth.
See the overview of [PyTrees](../../pytrees.md) or the documentation for the [`@pytree_node`](#archimedes.tree.struct.pytree_node) decorator for more details.

For example, let's say we have an algorithm that is composed of three discrete transfer functions, implemented as IIR filters:

```raw
                  |--> G --> y_g
u --> F --> y_f --|
                  |--> H --> y_h
```

Instead of maintaining six arrays (one each for `u` and `y` histories, across three filters), we can organize these into a logical hierarchical structure as follows:

In [ ]:
@struct.pytree_node
class FilterState:
    u_prev: np.ndarray
    y_prev: np.ndarray


@struct.pytree_node
class CompoundState:
    x_f: FilterState
    x_g: FilterState
    x_h: FilterState


@struct.pytree_node
class CompoundOutput:
    y_f: float
    y_g: float
    y_h: float


# Rewrite the IIR filter to operate with the structured state
def iir_filter(
    x: FilterState, u: float, b: np.ndarray, a: np.ndarray
) -> tuple[FilterState, float]:
    u_prev, y_prev = x.u_prev, x.y_prev

    # Update input history
    u_prev[1:] = u_prev[:-1]
    u_prev[0] = u

    # Compute output using the direct II transposed structure
    y = (np.dot(b, u_prev) - np.dot(a[1:], y_prev[: len(a) - 1])) / a[0]

    # Update output history
    y_prev[1:] = y_prev[:-1]
    y_prev[0] = y

    return FilterState(u_prev, y_prev), y


@arc.compile(return_names=("state_new", "y"))
def compound_filter(
    state: CompoundState, u: float
) -> tuple[CompoundState, CompoundOutput]:
    # For simplicity, just re-use the coefficients from the low-pass filter
    # we've already designed
    x_f, y_f = iir_filter(state.x_f, u, b, a)
    x_g, y_g = iir_filter(state.x_g, y_f, b, a)
    x_h, y_h = iir_filter(state.x_h, y_f, b, a)

    return CompoundState(x_f, x_g, x_h), CompoundOutput(y_f, y_g, y_h)


x0 = CompoundState(
    x_f=FilterState(u_prev=np.zeros(len(b)), y_prev=np.zeros(len(a) - 1)),
    x_g=FilterState(u_prev=np.zeros(len(b)), y_prev=np.zeros(len(a) - 1)),
    x_h=FilterState(u_prev=np.zeros(len(b)), y_prev=np.zeros(len(a) - 1)),
)
u0 = 1.0

args = (x0, u0)
x, y = compound_filter(*args)

pprint.pprint(y)

CompoundOutput(y_f=array(0.20657208),
               y_g=array(0.04267203),
               y_h=array(0.04267203))


In [ ]:
cleanup()  # Clean up any previous generated code
arc.codegen(compound_filter, args)

Let's take a look at the header to see what gets generated by this:

In [38]:
with open("compound_filter.h", "r") as f:
    c_code = f.read()

display_text(c_code)

```c

#ifndef COMPOUND_FILTER_H
#define COMPOUND_FILTER_H

#include "compound_filter_kernel.h"

#ifdef __cplusplus
extern "C" {
#endif

typedef struct {
    float u_prev[3];
    float y_prev[2];
} filter_state_t;

typedef struct {
    filter_state_t x_f;
    filter_state_t x_g;
    filter_state_t x_h;
} compound_state_t;

typedef struct {
    float y_f;
    float y_g;
    float y_h;
} compound_output_t;

// Input arguments struct
typedef struct {
    compound_state_t state;
    float u;
} compound_filter_arg_t;

// Output results struct
typedef struct {
    compound_state_t state_new;
    compound_output_t y;
} compound_filter_res_t;

// Workspace struct
typedef struct {
    long int iw[compound_filter_SZ_IW];
    float w[compound_filter_SZ_W];
} compound_filter_work_t;

// Runtime API
int compound_filter_init(compound_filter_arg_t* arg, compound_filter_res_t* res, compound_filter_work_t* work);
int compound_filter_step(compound_filter_arg_t* arg, compound_filter_res_t* res, compound_filter_work_t* work);


#ifdef __cplusplus
}
#endif

#endif // COMPOUND_FILTER_H
```

We see that the `FilterState` struct is automatically translated to `filter_state_t`, and the `CompoundState` and `CompoundOutput` structs are likewise translated to `compound_state_t` and `compound_output_t`.

These can be nested in one another just as in the Python code, so for example a `compound_state_t` is composed of three `filter_state_t`s

For now, you still need to manually copy the data from the output state back to the inputs (though [not for long](https://github.com/PineTreeLabs/archimedes/issues/77)), but now we can naturally access the hierarchical data in the C application:

```c
// Initialize as before
compound_filter_arg_t filter_arg;
compound_filter_res_t filter_res;
compound_filter_work_t filter_work;
compound_filter_init(&filter_arg, &filter_res, &filter_work);

// Set up the inputs to the function
filter_arg.u = read_sensor();

// Access nested states if necessary
check_state(&filter_arg.state.x_f);

// Evaluate the function numerically
compound_filter_step(&filter_arg, &filter_res, &filter_work);

// Do something with the outputs
handle_outputs(filter_res.y.y_g, filter_res.y.y_h);
```

When you call the `_step` function, all that happens is that the pointers to the underlying data are "marshalled" into a pointer array and passed to CasADi, amounting to minimal overhead compared to the gain in flexibility and readability for more complex functions.

With this approach you can create maintainable and scalable data types that match how you think about parameters, plant models, and control algorithms.

## Summary

In this final part of the hardware deployment tutorial, we covered the basic structure of the "interface" layer, or API, generated by Archimedes as a way to abstract application code from the low-level kernel implementation.

This Python-to-C code generation provides a convenient workflow for writing high-level logic in Python and then rapidly deploying to a range of C environments.

### Where to Go From Here

As mentioned, this code generation and hardware deployment workflow is an early proof of concept, but it is central to the Archimedes roadmap.

**If you're using Archimedes for hardware control and have questions, comments, or requests, please don't hesitate to reach out!**

In [22]:
cleanup()